# Flow Counter

* This notebook is used to define the flow_counter function.
* The flow_counter function runs throught our interpreted well-data and outputs the following: (per data set)
    * "flow_dict" which contains (unit:flow#) pairs
    * "flow_list" which can be appended to the OG dataset and indicates the flow#.
-----------------------------------------------------------------------------------------------------------------
# TODO:
* Maybe use some more generalization on the function by using regex expressions.
* Fix potentital bug for when the dataset  begins with volcaniclatic/siliciclastic facies.

## Importing libraries and data

In [1]:
# Importing libraries
import pandas as pd
import re

# Reading .csv
# data = pd.read_csv(r'Data/CSV/1-AV-001-PR.csv', sep=';') ## Dataset1
data = pd.read_csv(r'Data/CSV/1-CS-002-PR.csv', sep=';') ##Dataset2
data.head()

,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion
0,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),"3,65",NaN,NaN
1,1-CS-002-PR,Paranapanema,Simple lava (basic | L. crust),"1,12",NaN,NaN
2,1-CS-002-PR,Paranapanema,Siliciclastics,"1,52",NaN,NaN
3,1-CS-002-PR,Paranapanema,Simple lava (basic | U. crust),"11,6","37,28","0,650214592"
4,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),"24,24",NaN,NaN


## Flow Counter Algorithm/Function
-----------------------------------------------------------------------------------------------------------------   
* Things considered to indicate flow "change":
    * Upper crust of lava flow. 
    * Compound flows are counted as singular flow. Presence of compound flows indicate flow change.
    * Interbeds indicate flow changes.
    * Unit changes indicate flow changes.
-----------------------------------------------------------------------------------------------------------------    
* Output:
    * A flow_list list which contains the exact same number of values as the input Pandas DataFrame. (for appending)
    * A flow_dict containg (unit:flow#) pairs.

In [4]:
# Flow_counter function v.01

def flow_counter(data, flow_list, flow_dict):
    'Calculates the number os flows(flow#) per unit in a given dataframe.'
    'data --> Pandas DF containing at least unit;facies information.'
    'flow_list --> empty list to be populated with flow# values'
    'flow_dict --> empty dict to be populated with (unit:flow#) pairs.'
    
    # defining variables
    flow_counter = 0
    
    # Algorithm
    for i in range(len(data)):
        if i==0: # Setting up the first iteration
            flow_counter += 1
            flow_list.append(flow_counter)
        elif i == (len(data)-1): # Checking for the end of the list (working around out-of-bounds bug)
            if re.search(r'compound', data['Facies'].iloc[i], re.IGNORECASE) != None:
                flow_counter += 1
                flow_list.append(flow_counter)
                flow_dict[data['Unit'].iloc[i]] = flow_counter           
            else:
                flow_list.append(flow_counter)
                flow_dict[data['Unit'].iloc[i]] = flow_counter
            break      
        elif re.search(r'lava', data['Facies'].iloc[i]) != None: # If we're with a 'lava' facies
            if data['Unit'].iloc[i] == data['Unit'].iloc[i-1]: # If unit is the same as before
                if data['Facies'].iloc[i] == 'Simple lava (basic | U. crust)' or data['Facies'].iloc[i] == 'Compound lava (basic)': # Check for Simple lower crust OR compound flows:
                    flow_counter += 1
                    flow_list.append(flow_counter)
                    flow_dict[data['Unit'].iloc[i]] = flow_counter
                else:
                    flow_list.append(flow_counter)
                    #flow_dict[data['Unit'].iloc[i]] = flow_counter
            else: # If Unit changes
                if data['Unit'].iloc[i] in flow_dict: # If unit has already been added as a key to the flow_dict
                    flow_dict[data['Unit'].iloc[i-1]] = flow_counter
                    flow_counter = flow_dict[data['Unit'].iloc[i]]
                    flow_list.append(flow_counter)
                else: # If unit is NOT already associated with a value in the flow_dict
                    flow_dict[data['Unit'].iloc[i-1]] = flow_counter
                    flow_counter = 1
                    flow_dict[data['Unit'].iloc[i]] = flow_counter
                    flow_list.append(flow_counter)
                    # Check if the facies is an upper crust/compound lava
                    #if data['Facies'].iloc[i]  == 'Compound lava (basic)': # Working around the situtation where the unit changes into a "compound lava" facies.                   
                        #flow_counter += 1
                    #else:
                        #continue
        else: # Check for siliciclastic OR volcaniclastic; append a "-" for these units:
            if data['Unit'].iloc[i] == data['Unit'].iloc[i-1]: # If unit is the same as before
                flow_list.append('-')
            else:
                flow_counter = 0
                flow_list.append('-')
                

In [3]:
## TEST CELL
cs_pr_dict = {}
cs_pr_list = []
flow_counter(data,cs_pr_list,cs_pr_dict)
print(cs_pr_list)
print(cs_pr_dict)
data['flow#'] = cs_pr_list
data.head(20)

[1, 1, '-', 2, 2, 2, '-', 1, 1, 1, 2, 2, 2, 2, 2, 2, '-', 3, 4, 4, 4, 5, '-', 6, 6, 6, '-', 7, 7, 7, '-', 8, 8, 8, 8, '-', 8, 8, 8, 8, 8, 8, '-', 9, 9, 9, '-', 9, 9, 9, '-', 10, 10, 10, 11, 11, 11, 1, 2, 2, 2, '-', 3, 3, 3, '-', 3, 3, 3, '-', 3, 3, 3, 4, '-', 5, 5, 5, 6, 6, 6, 7, 7, 7, '-', 8, 8, 8, '-', 1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, '-', 6, 6, 6]
{'Paranapanema': 2, 'Pitanga': 11, 'Esmeralda': 8, 'Vale do Sol': 6}


,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion,flow#
0,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),"3,65",NaN,NaN,1
1,1-CS-002-PR,Paranapanema,Simple lava (basic | L. crust),"1,12",NaN,NaN,1
2,1-CS-002-PR,Paranapanema,Siliciclastics,"1,52",NaN,NaN,-
3,1-CS-002-PR,Paranapanema,Simple lava (basic | U. crust),"11,6","37,28","0,650214592",2
4,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),"24,24",NaN,NaN,2
5,1-CS-002-PR,Paranapanema,Simple lava (basic | L. crust),"1,44",NaN,NaN,2
6,1-CS-002-PR,Pitanga,Siliciclastics,"2,72",NaN,NaN,-
7,1-CS-002-PR,Pitanga,Simple lava (basic | U. crust),"7,12",32,"0,7175",1
8,1-CS-002-PR,Pitanga,Simple lava (basic | massive interior),"22,96",NaN,NaN,1
9,1-CS-002-PR,Pitanga,Simple lava (basic | L. crust),"1,92",NaN,NaN,1


## Others /// Notes: